In [4]:
from vmc_torch.fermion_utils import *
import symmray as sr
import quimb.tensor as qtn
import numpy as np

In [5]:
random_mps = generate_random_fmps(4, 4, Nf=4, seed=42)[0]
random_mps_1 = random_mps.copy()
config = [1,1,1,1]

tid = list(random_mps.ind_map['k1'])[0]
fts = random_mps[tid]
ftsdata = random_mps[tid].data
phys_ind_order = fts.inds.index('k1')
input_charge = 1
input_vec = np.array([1,0])
charge_sec_data_dict = ftsdata.blocks
# """charge_sec_data_dict = {(0,
#   0): array([[ 0.30471708, -1.03998411],
#         [ 0.7504512 ,  0.94056472]]),
#  (1,
#   1): array([[-1.95103519, -1.30217951],
#         [ 0.1278404 , -0.31624259]])}"""
new_fts_inds = fts.inds[:phys_ind_order] + fts.inds[phys_ind_order+1:]
ftsdata.blocks

{(0,
  0,
  0): array([[[-0.01680116, -0.85304393],
         [ 0.87939797,  0.77779194]],
 
        [[ 0.0660307 ,  1.12724121],
         [ 0.46750934, -0.85929246]]]),
 (0,
  1,
  1): array([[[ 0.36875078, -0.9588826 ],
         [ 0.8784503 , -0.04992591]],
 
        [[-0.18486236, -0.68092954],
         [ 1.22254134, -0.15452948]]]),
 (1,
  0,
  1): array([[[-0.42832782, -0.35213355],
         [ 0.53230919,  0.36544406]],
 
        [[ 0.41273261,  0.430821  ],
         [ 2.1416476 , -0.40641502]]]),
 (1,
  1,
  0): array([[[-0.51224273, -0.81377273],
         [ 0.61597942,  1.12897229]],
 
        [[-0.11394746, -0.84015648],
         [-0.82448122,  0.65059279]]])}

In [6]:
for charge_blk, data in charge_sec_data_dict.items():
    if charge_blk[phys_ind_order] == input_charge:
        print(data.shape)
        print(data @ input_vec)
        new_data = data @ input_vec
        # remove the charge_blk[phys_ind_order]
        new_charge_blk = charge_blk[:phys_ind_order] + charge_blk[phys_ind_order+1:]

new_duals = ftsdata.duals[:phys_ind_order] + ftsdata.duals[phys_ind_order+1:]
new_indinces = ftsdata.indices[:phys_ind_order] + ftsdata.indices[phys_ind_order+1:]
new_oddpos = (3*tid+1)*(-1)
new_charge = 1
new_charge_blk, new_data

new_charge_sec_data_dict = {new_charge_blk: new_data}
new_fts_data = sr.Z2FermionicArray.from_blocks(new_charge_sec_data_dict, duals = new_duals, charge=new_charge, oddpos=new_oddpos, symmetry=ftsdata.symmetry)

(2, 2, 2)
[[ 0.36875078  0.8784503 ]
 [-0.18486236  1.22254134]]
(2, 2, 2)
[[-0.42832782  0.53230919]
 [ 0.41273261  2.1416476 ]]


In [7]:
import pyinstrument
L=20
random_mps = generate_random_fmps(L, 6, Nf=L, seed=20)[0]
config = np.ones(L)
# config = np.array([1,1,1,1])
random_mps_1 = random_mps.copy()
charge_map = {0:0,1:1,2:1,3:0}
input_vec_map = {0:np.array([1,0]), 1:np.array([1,0]), 2:np.array([0,1]), 3:np.array([0,1])}
with pyinstrument.Profiler() as p:
    for n, site in zip(config, random_mps.sites):
        p_ind = random_mps.site_ind_id.format(site)
        tid = random_mps.sites.index(site)
        fts = random_mps[tid]
        ftsdata = random_mps[tid].data
        phys_ind_order = fts.inds.index(p_ind)
        charge = charge_map[n]
        input_vec = input_vec_map[n]
        charge_sec_data_dict = ftsdata.blocks
        new_fts_inds = fts.inds[:phys_ind_order] + fts.inds[phys_ind_order+1:]
        new_charge_sec_data_dict = {}
        for charge_blk, data in charge_sec_data_dict.items():
            if charge_blk[phys_ind_order] == charge:
                new_data = data @ input_vec
                new_charge_blk = charge_blk[:phys_ind_order] + charge_blk[phys_ind_order+1:]
                new_charge_sec_data_dict[new_charge_blk]=new_data
                
        new_duals = ftsdata.duals[:phys_ind_order] + ftsdata.duals[phys_ind_order+1:]

        if int(n) == 1:
            oddpos = (3*tid+1)*(-1)
        elif int(n) == 2:
            oddpos = (3*tid+2)*(-1)
        elif int(n) == 3 or int(n) == 0:
            oddpos = None
        # print(ftsdata.oddpos)
        
        new_fts_data = (-1)**(int(n)//3)*sr.Z2FermionicArray.from_blocks(new_charge_sec_data_dict, duals = new_duals, charge=charge, oddpos=oddpos, symmetry=ftsdata.symmetry)
        # if int(n) == 3:
        #     new_fts_data.apply_to_arrays(lambda x: -x)
        fts.modify(data=new_fts_data, inds=new_fts_inds, left_inds=None)


    amp1 = qtn.MatrixProductState(random_mps)
    print(amp1.contract())

# print(new_fts_data.indices, ftsdata.indices)
# p.print()

with pyinstrument.Profiler() as p:
    amp = random_mps_1.get_amp(config)
    # print(amp.contract())
# p.print()
amp.tensors[-1].data.indices

1751.9922062562068


(BlockIndex(chargemap={1: 3}, dual=True),)

In [10]:
random_mps._get_tids_from_inds('k1')

KeyError: 'k1'

In [8]:
with pyinstrument.Profiler() as p:
    amp = random_mps_1.get_amp(config, efficient=False)
    print(amp.contract())

p.print()

1751.9922062562068

  _     ._   __/__   _ _  _  _ _/_   Recorded: 02:58:19  Samples:  16
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.017     CPU time: 0.018
/   _/                      v4.7.3

Profile at /tmp/ipykernel_9815/597359715.py:1

0.017 <module>  ../../../../../tmp/ipykernel_9815/597359715.py:1
├─ 0.012 fMPS.get_amp  vmc_torch/fermion_utils.py:547
│  ├─ 0.011 TensorNetwork.contract  quimb/tensor/tensor_core.py:8438
│  │  └─ 0.011 TensorNetwork.contract_tags  quimb/tensor/tensor_core.py:8328
│  │     ├─ 0.010 wrapper  functools.py:883
│  │     │  └─ 0.010 tensor_contract  quimb/tensor/tensor_core.py:207
│  │     │     ├─ 0.008 array_contract  quimb/tensor/contraction.py:273
│  │     │     │  └─ 0.008 array_contract  cotengra/interface.py:735
│  │     │     │        [2 frames hidden]  cotengra
│  │     │     │           0.007 wrapper  functools.py:883
│  │     │     │           └─ 0.007 tensordot_fermionic  symmray/fermionic_core.py:762
│  │     │     │              ├─ 0.0

In [9]:
with pyinstrument.Profiler() as p:
    amp = random_mps_1.get_amp_efficient(config)
    print(amp.contract())

p.print()

1751.9922062562068

  _     ._   __/__   _ _  _  _ _/_   Recorded: 02:58:19  Samples:  7
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.008     CPU time: 0.008
/   _/                      v4.7.3

Profile at /tmp/ipykernel_9815/1071023817.py:1

0.007 <module>  ../../../../../tmp/ipykernel_9815/1071023817.py:1
├─ 0.006 MatrixProductState.contract  quimb/tensor/tensor_core.py:8438
│  └─ 0.006 MatrixProductState.contract_structured  quimb/tensor/tensor_1d.py:510
│     └─ 0.006 MatrixProductState.contract_cumulative  quimb/tensor/tensor_core.py:8552
│        └─ 0.006 MatrixProductState.contract_tags  quimb/tensor/tensor_core.py:8328
│           ├─ 0.005 wrapper  functools.py:883
│           │  └─ 0.005 tensor_contract  quimb/tensor/tensor_core.py:207
│           │     └─ 0.005 array_contract  quimb/tensor/contraction.py:273
│           │        └─ 0.005 array_contract  cotengra/interface.py:735
│           │              [3 frames hidden]  cotengra
│           │                 0.004 wrap

In [ ]:
for ts in amp1.tensors:
    print(ts.data.indices, ts.data.blocks)

(BlockIndex(chargemap={1: 3}, dual=False),) {(1,): array([-0.9141207 , -0.99843127, -0.05629192])}
(BlockIndex(chargemap={0: 3, 1: 3}, dual=True), BlockIndex(chargemap={0: 3, 1: 3}, dual=False)) {(0, 1): array([[-0.23527948,  0.17052219,  0.64643563],
       [ 0.43659917, -0.61343336,  0.47600246],
       [ 0.6501621 , -0.42597679, -0.48352906]]), (1, 0): array([[-1.74428461,  0.59557528,  0.16914278],
       [ 0.52018694,  1.29351947, -1.63057243],
       [ 1.8142719 ,  0.99733253, -2.4650048 ]])}
(BlockIndex(chargemap={0: 3, 1: 3}, dual=True), BlockIndex(chargemap={0: 3, 1: 3}, dual=False)) {(0, 1): array([[ 0.49015258, -2.36840306, -0.32175781],
       [ 1.68781126,  0.46888332, -0.71654015],
       [ 0.79306274, -1.65147665,  0.4393643 ]]), (1, 0): array([[-0.0916904 , -0.06519452, -1.93173922],
       [ 0.80764047,  0.10247752, -0.44591067],
       [ 1.6304031 ,  0.13477257,  0.45542962]])}
(BlockIndex(chargemap={1: 3}, dual=True),) {(1,): array([-0.1884522 , -0.15451561, -0.80172

In [ ]:
for ts in amp2.tensors:
    print(ts.data.indices, ts.data.blocks)

(BlockIndex(chargemap={1: 3}, dual=False),) {(1,): array([-0.9141207 , -0.99843127, -0.05629192])}
(BlockIndex(chargemap={0: 3, 1: 3}, dual=True), BlockIndex(chargemap={0: 3, 1: 3}, dual=False)) {(0, 1): array([[-0.23527948,  0.17052219,  0.64643563],
       [ 0.43659917, -0.61343336,  0.47600246],
       [ 0.6501621 , -0.42597679, -0.48352906]]), (1, 0): array([[-1.74428461,  0.59557528,  0.16914278],
       [ 0.52018694,  1.29351947, -1.63057243],
       [ 1.8142719 ,  0.99733253, -2.4650048 ]])}
(BlockIndex(chargemap={0: 3, 1: 3}, dual=True), BlockIndex(chargemap={0: 3, 1: 3}, dual=False)) {(0, 1): array([[ 0.49015258, -2.36840306, -0.32175781],
       [ 1.68781126,  0.46888332, -0.71654015],
       [ 0.79306274, -1.65147665,  0.4393643 ]]), (1, 0): array([[-0.0916904 , -0.06519452, -1.93173922],
       [ 0.80764047,  0.10247752, -0.44591067],
       [ 1.6304031 ,  0.13477257,  0.45542962]])}
(BlockIndex(chargemap={1: 3}, dual=True),) {(1,): array([-0.1884522 , -0.15451561, -0.80172

In [8]:
amp1.tensors[0].data, amp2.tensors[0].data

(Z2FermionicArray(shape~(3,):[+], charge=1, num_blocks=1),
 Z2FermionicArray(shape~(3,):[+], charge=1, num_blocks=1))

1751.9922062562068

  _     ._   __/__   _ _  _  _ _/_   Recorded: 00:41:57  Samples:  17
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.018     CPU time: 0.019
/   _/                      v4.7.3

Profile at /tmp/ipykernel_3051/775229312.py:1

0.017 <module>  ../../../../../tmp/ipykernel_3051/775229312.py:1
├─ 0.011 fMPS.get_amp  vmc_torch/fermion_utils.py:474
│  ├─ 0.010 TensorNetwork.contract  quimb/tensor/tensor_core.py:8438
│  │  └─ 0.010 TensorNetwork.contract_tags  quimb/tensor/tensor_core.py:8328
│  │     └─ 0.010 wrapper  functools.py:883
│  │        └─ 0.010 tensor_contract  quimb/tensor/tensor_core.py:207
│  │           ├─ 0.008 array_contract  quimb/tensor/contraction.py:273
│  │           │  └─ 0.008 array_contract  cotengra/interface.py:735
│  │           │        [1 frames hidden]  cotengra
│  │           │           0.008 wrapper  functools.py:883
│  │           │           └─ 0.008 tensordot_fermionic  symmray/fermionic_core.py:762
│  │           │              └─ 0.0